# Demo Version

In [8]:
#  General Prep  Made Here

import cv2
import time

from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np
# pip install tensorflow==2.12.1 

import statistics

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)
# Load the model
model = load_model("keras_model.h5", compile=False)
# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)


# Load the pre-trained Haar Cascade for face detection
cascade_path = 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(cascade_path)

# Check if the cascade file is loaded correctly
if face_cascade.empty():
    raise IOError(f"Failed to load cascade classifier from {cascade_path}")

# Create a video capture object
cap = cv2.VideoCapture(0)  # 0 for the default camera

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise ValueError("Unable to open video source")

# Define fixed window size for cropped face display
fixed_window_size = (224, 224)  # Width, Height

# Desired frame rate
desired_fps = 10
frame_duration = 1 / desired_fps  # Duration of each frame in seconds

# Define text parameters
text_size = 0.8  # Change this value to increase or decrease text size
text_color = (0, 255, 0)  # Text color in BGR format (Green)
text_thickness = 2  # Thickness of the text stroke
shadow_color = (0, 0, 0)  # Shadow color in BGR format (Black)
shadow_offset = (0, 0)  # Offset for shadow position

lastGuess=[]
text="text here"
guess="calculating"



try:
    while True:
        start_time = time.time()  # Start time of the loop

        # Capture frame-by-frame
        ret, frame = cap.read()  # ret is a boolean, frame is the captured image

        if not ret:  # Check if the frame was captured successfully
            print("Failed to capture image")
            break

        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Draw a rectangle around each face and crop the face
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            
            # Crop the face region
            face_crop = frame[y:y + h, x:x + w]

            # Resize the cropped face to the fixed window size
            face_crop_resized = cv2.resize(face_crop, fixed_window_size)

            # Add shadow text to the cropped face window
         
            font = cv2.FONT_HERSHEY_SIMPLEX
            # Calculate text size for shadow
            (text_width, text_height), baseline = cv2.getTextSize(text, font, text_size, text_thickness)
            text_x, text_y = 10, 30  # Position for the text
            shadow_x, shadow_y = text_x + shadow_offset[0], text_y + shadow_offset[1]

            ##########  
            # Replace this with the path to your image
            file_path="img.jpg"
            cv2.imwrite(file_path, face_crop_resized)
            image = Image.open(file_path).convert("RGB")

            # resizing the image to be at least 224x224 and then cropping from the center
            size = (224, 224)
            image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

            # turn the image into a numpy array
            image_array = np.asarray(image)

            # Normalize the image
            normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

            # Load the image into the array
            data[0] = normalized_image_array

            # Predicts the model
            prediction = model.predict(data)
            index = np.argmax(prediction)
            class_name = class_names[index]
            confidence_score = prediction[0][index]

            # Print prediction and confidence score
            print("Class:", class_name[2:], end="")
            print("Confidence Score:", confidence_score)
            

            #guess=f"{class_name[2:]}:{confidence_score}"

            ### make guess not change every frame
            if confidence_score>0.6:
                # add last guesses index
                lastGuess.append(index)
            
                # eğer son 10 tahminden fazla ise ilk indexi pırtlat
                if len(lastGuess)>9:
                    lastGuess.pop(0)
              
                resultIndex=int(statistics.median(lastGuess))
                print(resultIndex)
                class_name = class_names[resultIndex]
                guess=f"{class_name[2:]}:{confidence_score}"

        

            ##########
            # Show Result

            # Draw the shadow (background rectangle)
            cv2.rectangle(face_crop_resized, (shadow_x, shadow_y - text_height), (shadow_x + text_width, shadow_y + baseline), shadow_color, thickness=cv2.FILLED)
            # Draw the text
            cv2.putText(face_crop_resized, guess, (text_x, text_y), font, text_size, text_color, text_thickness, cv2.LINE_AA)

            # Display the cropped face in a separate window
            cv2.imshow('Cropped Face', face_crop_resized)
            # Ensure the window size is fixed
            cv2.resizeWindow('Cropped Face', fixed_window_size[0], fixed_window_size[1])

        # Add shadow text to the main video window
        main_text = 'Press "q" to exit'
        second_text = 'Second Line'
        (text_width, text_height), baseline = cv2.getTextSize(main_text, font, text_size, text_thickness)
        text_x, text_y = 10, 30  # Position for the text
        shadow_x, shadow_y = text_x + shadow_offset[0], text_y + shadow_offset[1]

        # Draw the shadow (background rectangle)
        cv2.rectangle(frame, (shadow_x, shadow_y - text_height), (shadow_x + text_width, shadow_y*2 + baseline), shadow_color, thickness=cv2.FILLED)
        # Draw the text
        cv2.putText(frame, main_text, (text_x, text_y), font, text_size, text_color, text_thickness, cv2.LINE_AA)
        
        # second text
        cv2.putText(frame, second_text, (text_x, text_y*2), font, text_size, text_color, text_thickness, cv2.LINE_AA)

        # Display the resulting frame in a window named 'Video'
        cv2.imshow('Video', frame)

        # Calculate the time taken for one loop iteration
        elapsed_time = time.time() - start_time
        # Calculate the delay needed to maintain the desired frame rate
        delay = max(1, int((frame_duration - elapsed_time) * 1000))
        
        # Break the loop if 'q' key is pressed
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    pass
finally:
    # Release the capture and close windows
    cap.release()
    cv2.destroyAllWindows()


1/1 [==============================] - 1s 690ms/step
Class: focused
Confidence Score: 0.9847476
0
1/1 [==============================] - 0s 25ms/step
Class: focused
Confidence Score: 0.8365187
0
1/1 [==============================] - 0s 26ms/step
Class: focused
Confidence Score: 0.96351784
0
1/1 [==============================] - 0s 25ms/step
Class: focused
Confidence Score: 0.9347937
0
1/1 [==============================] - 0s 24ms/step
Class: focused
Confidence Score: 0.9852579
0
1/1 [==============================] - 0s 24ms/step
Class: focused
Confidence Score: 0.94140565
0
1/1 [==============================] - 0s 22ms/step
Class: focused
Confidence Score: 0.8266407
0
1/1 [==============================] - 0s 24ms/step
Class: focused
Confidence Score: 0.96705765
0
1/1 [==============================] - 0s 25ms/step
Class: focused
Confidence Score: 0.9625024
0
1/1 [==============================] - 0s 22ms/step
Class: focused
Confidence Score: 0.9543104
0
1/1 [=====================